In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd 

In [2]:
# set executable path 
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/Users/italiathe2nd/.wdm/drivers/chromedriver/mac64/92.0.4515.107/chromedriver] found in cache


In [3]:
# set up the url for scraping
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
# searching for elements with a specific combination of tag (div) and attribute (list_text) 
# In the inspect page it looks like this: <div class="list_text">
# we're also telling browser to wait one second before searching components
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# set up the html parser
# div.list_text in assigned slide_elem to look for <div /> tag and other tags within the <div /> element.
# slide_elem is the parent element
# the period in div.list_text is used for selecting the class vwe want, in this case it's list_text 
# CSS works from right to left, such as returning the last item on the list instead of 
# the first. Because of this, when using select_one, the first matching element returned will 
#be a <li /> element with a class of slide and all nested elements within it.
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [5]:
# begin scraping 
# chained .find to previously assigned variable 
# Basically syaing: " here's a lot of info here, look an find this specific data" - ('div', class=_'content_title')
slide_elem.find('div', 'div', class_='content_title')


<div class="content_title">NASA's Mars 2020 Heads Into the Test Chamber</div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`
# added .get_text() to .find() to get only the text of the element, in this case justthe titles  
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Mars 2020 Heads Into the Test Chamber"

In [7]:
# Use the parent element to find the paragraph text
# using .find(0 )
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'In this time-lapse video taken at JPL, engineers move the Mars 2020 rover into a large vacuum chamber for testing in Mars-like environmental conditions.'

### Featured Images

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# Find and click the full image button
# full_image_elem is a new variable to hold the scraping resu;t 
# browser.find_by_tag('button') is to find an element by its tag
# full_image_elem.click() makes Splinter 'click' the image to view its full size 
# [] is the indexing chained at the end of the first line of code - means we want 
# the browser to click 2nd button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [11]:
# Find the relative image url
# An img tag is nested within this HTML, so we've included it.
# .get('src') pulls the link to the image
# this tells beautiful soup to look inside <img /> tag for an image with a class of fancybox=image
# "this is where the image we want lives- use the link that's inside of these tags"
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [12]:
# add the first part of the URL to our code
# Use the base URL to create an absolute URL
# the variable img_url holds our f-string
# the f-string is a type of string formatting used to print statements in Python 
# the curly brackets hold a variable that will be inserted into the f-string when it's executed 
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [13]:
#scrape entire table with Pandas' .read_html() function

# create a new DataFrame from the HTML table 
# .read_html() specifically searches for and returns a list of tables found in HTML
# [0] index tells pandas to pull only the first table it encounters then turn it into a dataframe
df = pd.read_html('https://galaxyfacts-mars.com')[0]
# assign columns to the new dataframe for clarity 
df.columns=['Description', 'Mars', 'Earth']
# .set_index() function turns the description column into the dataframe's index
# inplace=True means that the updated index wil remain in place, w/o having to assing the DF to a new variable 
df.set_index('Description', inplace=True)
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
# this code converts the DataFrame back intop HTML-ready code 
# returns a <table /> element with a lot of nested elements 
# adding this exact code to the web app, the data it stores will be presented in an easy to read tabular format
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [15]:
# this line ends the automated browser session 
# we only want it active while we're scraping
browser.quit()

In [16]:
# The next step in making this an automated process is to download the
#current code into a Python file. It won't transition over perfectly, we'll 
#need to clean it up a bit, but it's an easier 
#task than copying each cell and pasting it over in the correct order.